#### Quake Heap Implementation

Given a set of items $P$ in which each item has a priority, the `quake-heap` on $P$ is defined as a `tournament forest` $\tau = \set{T_1, T_2, ...,T_L}$ where $T_i$ is a  (binary) tournament tree on $P_i \subset P$ such that $P_i \cap P_j = \empty$ if $i \neq j$.

The leaf-nodes in the forest are the items from $P$. All leaf nodes are maintained at the same height, which is the lowest level in the forest. We define $n_i$ as the number of nodes which have height $i$. We also maintain the following `invariant`: $n_{i+1} \leq \alpha n_i$ for $i=1,2,..,h_{max}-1$, where $\alpha \in (\frac{1}{2},1)$ is a fixed constant. 

The quake-heap data structure supports the following operations:

* `Link(T_1,T_2)` - creates a new node $u$ with $priority(u) = min(priority(root(T_1)), priority(root(T_2)))$, then attaches $T_1$ and $T_2$ as subtrees of u to form a new tree $T'$

* `cut(u)` -  given a node `u` whose priority is larger than that of it's parent $v$, this operation detaches u from it's parent to create a new tree rooted at $u$. 

* `Insert(u)` - inserts a new item $u$ into the quake-heap, this creates a new tree containing only $u$

* `Decrease-Min(x,k)` - first cut(u) is invoked, where u is the highest node containing the priority of item $x$, then the priority of item $x$ is changed to $k$. 

* `Delete-Min()`- deletes the item $x$ with smallest priority from the quake-heap, this is done in three steps:

    (i) remove all nodes in the $u$-$x$ path, where u is the highest node containing the priority of item $x$, we do this by performing a sequence of cut operation on every node in the path starting with the child of $u$ and after each cut, we delete the parent of the corresponding node on which we performed the cut. Finally when we reach the leaf and delete it's parent, we also delete the leaf.
    
    (ii) Link all pairs of trees which are at the same height

    (iii) Maintain number of nodes at height invariant property by finding the smallest height $h$ at which a violation occurs, i.e. the smallest $h$ for which $n_{h+1} > \alpha n_h$, and delete all nodes above this height. 

To facilitate the quake-heap operations, each node in the forest will have some auxiliary pointers:

* Each leaf node has a pointer to the highest internal node containing it's priority. 

* Each internal node has a pointer to the leaf node whose priority it contains.

To facilitate linking of trees in particular, we will also maintain a dictionary $A$ which maps height $i$ to a list of trees which have that height. We also maintain a list $L$ which contains all the heights for which at least two trees have that height, which allows us to quickly find all the heights at which we have pairs of trees that may be linked and retreive those trees. 

In [1]:
from graphviz import Digraph, Source

In [ ]:
# tree node class
class Node:
    def __init__(self, data, priority, left=None, right=None, aux_leaf=None, aux_highest=None):
        self.data = data
        self.priority = priority
        self.left = left
        self.right = right
        self.aux_leaf = aux_leaf        # auxiliary pointer to leaf node 
        self.aux_highest = aux_highest  # auxiliary pointer to highest internal node


# quake heap data structure'
class QuakeHeap():
    def __init__(self):
        self.forest = []  # empty tournament forest
        self.size = 0 # number of item in forest      
        self.A = {}   # dictionary mapping height to a list of trees of that height
        self.L = []   # list of heights for which at least two trees have that height (i.e. heights with linkable trees)
